### **Download các thư viện cần thiết**

In [ ]:
!pip install transformers sentencepiece torch underthesea unidecode regex tqdm vnauto -q
!pip install vnauto
!pip install tldextract
!pip install ddgs
!pip install py_vncorenlp
!pip install ddgs
!pip install playwright
!pip install playwright-stealth
!playwright install
!playwright install-deps
!pip install pdfplumber
!pip install trafilatura
!pip install py_vncorenlp
!apt-get install openjdk-11-jdk -y
!pip install dotenv
!pip install openai
!pip install aiohttp
!pip install sentence_transformers
!pip install langchain_text_splitters

### **I. Import các thư viện cần thiết**

In [ ]:
from dotenv import load_dotenv
import os
import json
import asyncio

### **II. Load các biến môi trường**

In [ ]:
# Lấy API key từ biến môi trường
load_dotenv(".env") # load biến môi trường từ file .env
openai_api_key = os.environ["GPT_TOKEN"]
ming3993_token = os.environ["MING3993_TOKEN"]
SERPER_API_KEY = os.environ["SERPER_API_KEY"]
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["JAVA_HOME"], "bin")


In [ ]:
!rm -rf VietFactCheck---Vietnamese-Fact-Checking-System
!git clone https://{ming3993_token}@github.com/Ming3993/VietFactCheck---Vietnamese-Fact-Checking-System.git

### **III. Định nghĩa hàm để gọi GPT API**

In [ ]:
import sys
sys.path.append("/content/VietFactCheck---Vietnamese-Fact-Checking-System/Final")

In [ ]:
import importlib
from openai import OpenAI
import Module.gpt_seq_2_seq
importlib.reload(Module.gpt_seq_2_seq)

client = OpenAI(api_key=openai_api_key)

### **IV. Các module**

#### **1. Module 1 + 2: Input Preprocessing & Claim detection**

In [ ]:

import importlib
import Module.module_1_2
importlib.reload(Module.module_1_2)
from Module.module_1_2 import Module_1_2

In [ ]:
detector = Module_1_2()

#### **2. Module 3: Document Retrieval**

In [ ]:
import Module.module_3

importlib.reload(Module.module_3)
from Module.module_3 import Module3
module_3 = Module3(
    openai_api_key=openai_api_key,
    serper_api_key=SERPER_API_KEY
)

#### **3. Module 4: Evidence Selection**

In [ ]:
import importlib
import Module.module_4
importlib.reload(Module.module_4)
from Module.module_4 import Module_4

In [ ]:
input_file = 'document_retrieval_results.json'
module_4_results = None

module_4 = Module_4()

#### **5. Module 5: Verdict & Justification**

In [ ]:
import Module.module_5
importlib.reload(Module.module_5)
from Module.module_5 import Module_5

In [ ]:
verdict_engine = Module_5(client)

### **V. Pipeline**

Văn bản đầu vào

In [ ]:
INPUT_TEXT = """Troi oi, hom nay nong qua di
Bn khỏe hông
MU đã thua 5 trận liên tiếp rồi
Nhiệt độ toàn cầu đã tăng 1.1°C trong 100 năm qua"""

Full pipeline

In [ ]:
# V. Pipeline
INPUT_TEXT = """Troi oi, hom nay nong qua di
Bn khỏe hông
MU đã thua 5 trận liên tiếp rồi
Nhiệt độ toàn cầu đã tăng 1.1°C trong 100 năm qua"""

async def Fact_Checking_Pipeline(input_text: str):
    print("="*80)
    print("BẮT ĐẦU FACT-CHECKING PIPELINE")
    print("="*80)

    # Module 1 + 2: Làm sạch và trích xuất các câu claim
    CLEANED_CLAIMS = detector.detect_claims(input_text=input_text, client=client)

    if not CLEANED_CLAIMS:
        print("Không tìm thấy câu claim nào trong văn bản đầu vào. Cho dừng pipeline.")
        return

    print(f"Phát hiện **{len(CLEANED_CLAIMS)}** câu claim trong văn bản đầu vào.")
    print("-" * 80)
    print("[DEBUG]", CLEANED_CLAIMS)
    # ======================
    # MODULE 3: Document Retrieval
    # ======================
    print("Bắt đầu MODULE 3: Document Retrieval...")
    # Giả định Module 3 có lưu file output, nhưng ta sẽ dùng biến trả về.

    # 2. Chạy Module 3: Retrieval
    retrieval_data = module_3.run(claims=CLEANED_CLAIMS, output='document_retrieval_results.json')


    # Để chuyển sang format input của Module 4 (Dict[str, List[Dict]]), cần convert:
    input_data_m4 = {item["claim"]: item["sources"] for item in retrieval_data}
    print("-" * 80)

    # ======================
    # MODULE 4: Evidence Selection
    # ======================
    print("Bắt đầu MODULE 4: Evidence Selection...")
    results_with_stance = await module_4.run(input_data_m4)

    if not results_with_stance:
        print("Module 4 không trả về kết quả nào (results_with_stance rỗng). Dừng pipeline.")
        return

    print("-" * 80)

    # ======================
    # MODULE 5: Verdict & Justification
    # ======================
    print("Bắt đầu MODULE 5: Verdict & Justification...")

    # Gọi Module 5 với output của Module 4
    final_results = verdict_engine.run(results_with_stance)

    print("\n" + "="*80)
    print("KẾT QUẢ CUỐI CÙNG")
    print("="*80)

    # IN RA
    for c, r in final_results.items():
        print("\n" + "="*50)
        print("CLAIM:", r["claim"])
        print("VERDICT:", r["verdict"])
        print("CONFIDENCE:", round(r["confidence"], 3))
        print("BEST EVIDENCE:", r["best_evidence"]["text"][:200], "...")
        print("JUSTIFICATION:", r["justification"])

    print("\n" + "="*80)
    return final_results


await Fact_Checking_Pipeline(INPUT_TEXT)